# EXAMEN LAB01 - Clasificación multiclase

###En el dataset "poker-game-dataset" se va a predecir la mano que tiene el Poker, donde tenemos 10 Clases de 0 al 9. Se aplicacrá la regresión logística onevsall. Se comenzará haciendo el preprocesamiento del dataset con Pandas.

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. Librerías

In [68]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

#Libreria pandas y scikitlearn para el preprocesamiento de datasets
import pandas as pd
import sklearn as skl

#2. Contenido del dataset
En primer lugar, el contenido del dataset se carga en "dataframe", el cual Panda utilizará como unidad de información.\
El conjunto de datos consta de dos archivos:

mnist_train.csv\
mnist_test.csv\
El archivo mnist_train.csv contiene 60.000 ejemplos y etiquetas de entrenamiento. mnist_test.csv contiene 10.000 ejemplos de prueba y etiquetas. Cada fila consta de 785 valores: el primer valor es la etiqueta (un número del 0 al 9) y los 784 valores restantes son los valores de los píxeles (un número del 0 al 255).

In [69]:
dataframe = pd.read_csv('/content/drive/MyDrive/SIS 420/EXLAB/poker-hand-training.csv')
print(dataframe)
print(dataframe.describe())

       Suit of Card 1  Rank of Card 1  Suit of Card 2  Rank of Card 2  \
0                   1              10               1              11   
1                   2              11               2              13   
2                   3              12               3              11   
3                   4              10               4              11   
4                   4               1               4              13   
...               ...             ...             ...             ...   
85005               4               1               4               7   
85006               3               3               1               4   
85007               4               7               4              11   
85008               2              13               2               4   
85009               2              13               1               7   

       Suit of Card 3  Rank of Card 3  Suit of Card 4  Rank of Card 4  \
0                   1              13             

#3.Preprocesamiento del dataset

###3.1. Preprocesamiento del dataset: Separar atributos(X) de la etiqueta (y) que se va a usar para el entrenamiento
Los atributos(X) se van a cargar en la variable "dataset" y la etiqueta "y" en la variable "label".

In [70]:
dataset = dataframe.drop("Poker Hand", axis=1)
label = dataframe["Poker Hand"].copy()

###3.2. Preprocesamiento del dataset: Atributos con texto
En nuestro dataset no tenemos atributos NO NUMÉRICOS.

###Limpieza de valores NaN en los atributos para evitar que los valores de theta salgan cero

In [71]:
print(dataframe.describe())

       Suit of Card 1  Rank of Card 1  Suit of Card 2  Rank of Card 2  \
count    85010.000000    85010.000000    85010.000000    85010.000000   
mean         2.504482        6.991519        2.500659        7.036725   
std          1.117032        3.752044        1.118272        3.752609   
min          1.000000        1.000000        1.000000        1.000000   
25%          2.000000        4.000000        2.000000        4.000000   
50%          3.000000        7.000000        2.000000        7.000000   
75%          4.000000       10.000000        4.000000       10.000000   
max          4.000000       13.000000        4.000000       13.000000   

       Suit of Card 3  Rank of Card 3  Suit of Card 4  Rank of Card 4  \
count    85010.000000    85010.000000    85010.000000    85010.000000   
mean         2.504105        7.008352        2.495095        6.976062   
std          1.118622        3.739830        1.116682        3.745265   
min          1.000000        1.000000        1.000

In [72]:
# Suponiendo que 'dataframe' es tu DataFrame
# Itera sobre cada columna y verifica si hay valores NaN
for col in dataframe.columns:
    # Verifica si hay valores NaN en la columna actual
    if dataframe[col].isnull().sum() > 0:
        # Imprime el nombre de la columna y la cantidad de valores NaN
        print(f"Categoría '{col}' tiene {dataframe[col].isnull().sum()} valores NaN.")

###Observación: No hay valores nan

## 4. Aplicación de la regresión logística onevsall
###Al inicio cargamos el dataframe en data.


In [73]:
# La entrada es de 785 elemnetos contando el X0
input_layer_size  = 11     #Contando X0

# 10 etiquetas, de 0 a 9
num_labels = 10

#  datos de entrenamiento almacenados en los arreglos X, y
data = dataframe.copy()

# print(data['y'])
# print(data['y'].ravel())
# print(data['X'])
#X = data[:, :25]                                                #array de NumPy
#y = data[:, 25]

#Dataframe de Pandas
X = data.iloc[:, 0:10]  # Seleccionar todas las filas y las columnas de 1 a 784.
y = data.iloc[:, 10]   # Seleccionar todas las filas y solo la columna 10.

# X, y = data['X'], data['y'].ravel()
# print(X)
# print(y)
# establecer el dígito cero en 0, en lugar del 10 asignado a este conjunto de datos
# Esto se hace debido a que el conjunto de datos se utilizó en MATLAB donde no hay índice 0
#y[y == 10] = 0
#print(y)

m = y.size

In [74]:
#print(X[0,:])
#print(y)

#Dataframe de Pandas
print(X.iloc[0, :])               # Imprimir la primera fila  de X,todas las columnas
print(y)                          # Imprimir la columna 'y' completa

Suit of Card 1     1
Rank of Card 1    10
Suit of Card 2     1
Rank of Card 2    11
Suit of Card 3     1
Rank of Card 3    13
Suit of Card 4     1
Rank of Card 4    12
Suit of Card 5     1
Rank of Card 5     1
Name: 0, dtype: int64
0        9
1        9
2        9
3        9
4        9
        ..
85005    3
85006    1
85007    1
85008    0
85009    1
Name: Poker Hand, Length: 85010, dtype: int64


###Normalización

In [75]:
def featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)

    # Manejar el caso de desviación estándar igual a cero
    sigma[sigma == 0] = 1  # Si la desviación estándar es cero, establecerla a 1

    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [76]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [77]:
print(X_norm)

       Suit of Card 1  Rank of Card 1  Suit of Card 2  Rank of Card 2  \
0           -1.346865        0.801829       -1.341952        1.056145   
1           -0.451630        1.068352       -0.447710        1.589110   
2            0.443605        1.334875        0.446532        1.056145   
3            1.338840        0.801829        1.340774        1.056145   
4            1.338840       -1.596877        1.340774        1.589110   
...               ...             ...             ...             ...   
85005        1.338840       -1.596877        1.340774       -0.009787   
85006        0.443605       -1.063831       -1.341952       -0.809235   
85007        1.338840        0.002260        1.340774        1.056145   
85008       -0.451630        1.601398       -0.447710       -0.809235   
85009       -0.451630        1.601398       -1.341952       -0.009787   

       Suit of Card 3  Rank of Card 3  Suit of Card 4  Rank of Card 4  \
0           -1.344613        1.602127       -1.338

In [78]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
#X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)                          #LO COMENTAMOS PORQUE ABAJO ESTAMOS VOLVIENDO A AGREGAR UNOS -> EN LA FUNCIÓN ONEVSALL
X = X_norm                                                                      #Y HACEMOS X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

### Vectorización de la función de costo

In [79]:
def sigmoid(z):                                                                 #SIGMOIDE
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [80]:
def lrCostFunction(theta, X, y, lambda_):                                       #FUNCIÓN DE COSTO
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

### Clasificacion One-vs-all   (con vectorización regularizada)


In [81]:
def oneVsAll(X, y, num_labels, lambda_):                                        #FUNCIÓN ONEVSALL
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
                                                                                #AHORA EMPIEZA ONE vs ALL->TENEMOS QUE HACER UNA BINARIA POR CADA CLASE Y CALCULAR SUS THETAS:
    for c in np.arange(num_labels):                                             #VA A HACER EN c DESDE 0 HASTA EL num_labels -> TENEMOS 10 etiuetas (0,1,2,3,4,5,6,7,8,9)
        initial_theta = np.zeros(n + 1)                                         #POR CADA etiqueta VA A TENER UN THETA INICIAL QUE ESTA PONIENDO EN ZEROS
        options = {'maxiter': 200}            #50, 100, 200
        res = optimize.minimize(lrCostFunction,                                 #EN res ESTOY APLICANDO LA FUNCIÓN DE optimize.minimize. LE PASO LA FUNCIÓN DE COSTO,
                                initial_theta,                                  # LOS VALORES INICIALES DE THETA,
                                (X, (y == c), lambda_),                         # Y LAS VARIABLES RESTANTES QUE USA THETA: X, y(YA SEA 0,1,2....Y calcule en función a ello los valores de theta), lambda.
                                jac=True,                                       # jac = True  -> TIENE QUE ESTAR EN TRUE
                                method='CG',                                    # EL MÉTODO PUEDE SER EL DESCENSO POR EL GRADIENTE ESTOCÁSTICO, EL CG
                                options=options)                                # EN options=options solo está indicando el max numero de iteraciones

        all_theta[c] = res.x

    return all_theta

In [82]:
lambda_ = 0.0001                                                                #0.01, 0.001
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 11)


In [83]:
print(all_theta)

[[ 5.46985284e-03  7.50589006e-03 -1.51110958e-02  1.44494167e-02
   3.45407415e-03  6.38042177e-04 -5.85232426e-03 -6.11079580e-03
  -1.01774766e-02  1.25068619e-02  4.30697541e-03]
 [-3.19129422e-01 -1.33291548e-02  9.50473522e-03 -1.97151686e-02
   2.31636293e-03  6.38891982e-04  5.50314874e-03  3.68591698e-03
   4.31784113e-03 -9.83517128e-03 -1.27684677e-03]
 [-2.97278874e+00  2.63659006e-02  2.92278958e-02  2.16603916e-02
  -3.83970798e-02 -1.56763224e-02 -1.22677691e-02  1.52794608e-02
   2.25587395e-02 -2.12000781e-02 -1.23292868e-02]
 [-3.82406826e+00 -5.18819795e-03 -1.15033086e-02 -5.67392832e-03
  -6.49979467e-03 -1.57485120e-03  3.77142065e-03  1.23750199e-02
  -1.08614659e-02  6.76420907e-03 -2.86832762e-02]
 [-5.57294938e+00  4.80755247e-03  5.20435980e-02  2.91917953e-02
   1.49151501e-01  5.32552733e-02  1.19207879e-01 -1.69111200e-02
   1.11776995e-01  1.20475385e-02  1.10996579e-01]
 [-6.32955764e+00  8.26079403e-02 -4.31943910e-02  8.58305893e-02
  -4.05807595e-02  


#### Prediccion One-vs-all

In [84]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

###Precisión del conjunto de entrenamiento

In [85]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)                                                        #LE PASAMOS LOS VALORES DE THETA Y LE PASAMOS EL VALOR DE X
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))

(85010, 10)
Precision del conjuto de entrenamiento: 50.14%


###PREDICCIONES

In [86]:
# Función de normalización
def featureNormalize(X):
    X_norm = X.copy()
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)

    # Manejar el caso de desviación estándar igual a cero
    sigma[sigma == 0] = 1  # Si la desviación estándar es cero, establecerla a 1

    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

# Cargar el dataset mnist_test.csv
poker_test = pd.read_csv('/content/drive/MyDrive/SIS 420/EXLAB/poker-hand-testing.csv')

# Obtener las características (columnas 0 a 9) y las etiquetas (columna 10)
XPrueba = poker_test.iloc[:, :10].values
yPrueba = poker_test.iloc[:, 10].values

# Normalizar las características
XPrueba, mu, sigma = featureNormalize(XPrueba)

# Agregar la columna de unos para el término de sesgo (bias)
XPrueba = np.concatenate([np.ones((XPrueba.shape[0], 1)), XPrueba], axis=1)

# Realizar predicciones
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)
print(p)  # Predicciones

# Mostrar los valores verdaderos
print(yPrueba)  # Etiquetas verdaderas

[0 0 0 ... 0 0 0]
[0 1 1 ... 1 1 2]


###Precisión del Conjunto de Prueba

In [88]:
# Cargar el dataset mnist_test.csv
poker_test = pd.read_csv('/content/drive/MyDrive/SIS 420/EXLAB/poker-hand-testing.csv')

# Obtener las características (columnas 0 a 9) y las etiquetas (columna 10)
XPrueb = poker_test.iloc[:, 0:10].values
yPrueb = poker_test.iloc[:, 10].values

print(XPrueb.shape)
pred = predictOneVsAll(all_theta, XPrueb)                                                        #LE PASAMOS LOS VALORES DE THETA Y LE PASAMOS EL VALOR DE X
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == yPrueb) * 100))

(1000000, 10)
Precision del conjuto de entrenamiento: 0.06%
